In [ ]:
#  ╔═══╦═════════╗
#  ║ 1 ║ SET UP  ║
#  ╚═══╩═════════╝


# Dash packages
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State, ALL
import plotly.graph_objects as go
import dash_leaflet as dl
from dash_leaflet import express as dlx


# Data packages
import pandas as pd
import json


# Aux packages
import kpi_heatmaps as kh
from aux_functions import lista_ufs
    

# Paleta de cores
marks = [0, 10, 20, 30, 60, 70, 80, 90]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026']


# Indicadores
style_functions = {
    'Lixo Coletado': kh.get_lixo,
    'Energia Elétrica': kh.get_energia,
    'Rede Geral de Abastecimento de Água': kh.get_agua,
    'Sem Banheiro ou Sanitário': kh.get_sanitario
}


# Listas
KPIS = list(style_functions.keys())
UFS = [uf for uf in lista_ufs()]


def get_info(feature=None, kpi=None, uf='SC'):
    header = [html.H4(lista_ufs()[uf]['Nome'])]
    if not feature:
        return header + ["Escolha um município"]
    return header + [html.B(feature["properties"]["name"]), html.Br(),
                     f'{kpi}: {float(feature["properties"][kpi])}%']




#  ╔═══╦════════════╗
#  ║ 2 ║ FRONT END  ║
#  ╚═══╩════════════╝


# Criar Dash App
app = dash.Dash(external_stylesheets=[dbc.themes.SANDSTONE])
app.title = 'Indicadores Municipais'
app.layout = html.Div([
    
    dcc.Store('memory_kpi', data='Lixo Coletado'),
    
    dbc.NavbarSimple([
        dcc.Dropdown(
            options = [{'label': uf, 'value': uf} for uf in UFS],
            value = 'SC',
            id = 'uf_dropdown'
        ),
        dbc.DropdownMenu(
            [dbc.DropdownMenuItem(kpi, id = {'item_kpi': kpi}) for kpi in KPIS],
            direction = 'left',
            nav = True,
            in_navbar = True,
            label = 'Indicador',
            id = 'button_kpi'
        )
    ],
    brand = "Mapa de Indicadores Municipais",
    brand_href = "#",
    color = "primary",
    dark = True,
),
    
    dbc.Container([
        dbc.Row(style={'width': '100%', 'height': '90vh', 'margin': "auto", "display": "block"}, id="map")
    ])
    
])




#  ╔═══╦═════════╗
#  ║ 3 ║ SERVER  ║
#  ╚═══╩═════════╝


# Carregar mapa
@app.callback([Output('map', 'children'),
               Output('button_kpi', 'label'),
               Output('memory_kpi', 'data')],
              [Input({'item_kpi': ALL}, 'n_clicks')],
              [State('uf_dropdown', 'value')],
               prevent_initial_call = True)
def set_kpi(click, uf):
    
    # Identificar botão apertado
    kpi = dash.callback_context.triggered[0]['prop_id'][13:-11]
    if kpi == '':
        kpi = KPIS[0]
        
    # Baixar JSON
    with open(f'assets/{uf}_geo.json', 'r') as js:
        geo = json.load(js)
    
    # Criar colorbar
    ctg = ["{}+".format(mark, marks[i + 1]) for i, mark in enumerate(marks[:-1])] + ["{}+".format(marks[-1])]
    colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
    
    # Criar geojson
    options = {'hoverStyle': {'weight': 5, 'color': '#666', 'dashArray': ''}, 'zoomToBoundsOnClick': True}
    geojson = dlx.geojson(geo, id="geojson", defaultOptions=options, style=style_functions[kpi])
    #geojson = dl.GeoJSON(geo, id="geojson", defaultOptions=options, featureStyle=feature_style, featureId=feature_id)
    
    # Criar caixa de informações
    info = html.Div(children=get_info(), id="info", className="info",
                    style={"position": "absolute", "bottom": "10px", "right": "10px", "z-index": "1000"})
    
    return dl.Map(
        children = [dl.TileLayer(), geojson, colorbar, info],
        center = [lista_ufs()[uf]['Latitude'], lista_ufs()[uf]['Longitude']],
        zoom = 7.3), kpi, kpi


# Atualizar caixa de informações
@app.callback(Output("info", "children"),
             [Input("geojson", "featureHover")],
             [State('memory_kpi', 'data'),
              State('uf_dropdown', 'value')])
def info_hover(feature, data, uf):
    return get_info(feature, kpi=data, uf=uf)


# Rodar server
if __name__ == '__main__':
    app.run_server()